In [1]:
import matplotlib
matplotlib.use('Agg')
import matplotlib.pyplot as plt

import numpy as np
np.random.seed(123)

import random
random.seed(123)

import tensorflow as tf
from keras.backend.tensorflow_backend import set_session
config = tf.ConfigProto()
config.gpu_options.per_process_gpu_memory_fraction = 0.5
set_session(tf.Session(config=config))

from keras.utils import plot_model
from keras.models import Sequential, load_model
from keras.layers import Dense, Activation, Flatten, BatchNormalization, Dropout, Reshape
from keras.optimizers import Adadelta
from keras.callbacks import TensorBoard, EarlyStopping, ModelCheckpoint
from sklearn.preprocessing import label_binarize
import cv2
import pdb
import progressbar
import os
from scipy import stats
from datasets import get_dataset

Using TensorFlow backend.


In [2]:
ds = get_dataset("bupa", 0.7, 0.25)
[L_x, L_y], U, [test_x, test_y] = ds.get_data()
U = np.array(U)
n_views = 5
views = []
for ind in range(n_views):
    views.append([np.array(L_x[ind * n_views:(ind + 1) * n_views]), np.array(L_y[ind * n_views:(ind + 1) * n_views])])
for ind in range(n_views):
    print views[ind][0].shape, views[ind][1].shape

(5, 6) (5, 2)
(5, 6) (5, 2)
(5, 6) (5, 2)
(5, 6) (5, 2)
(5, 6) (5, 2)


In [4]:
# Define Models
models = []
n_attr = views[ind][0].shape[1]
for ind in range(n_views):
    model = Sequential()
    model.add(Dense(input_shape=(n_attr,), units=n_attr / 2))
    model.add(Activation('relu'))
    model.add(Dense(units=n_attr/5))
    model.add(Activation('relu'))
    model.add(Dense(units=views[ind][1].shape[1]))
    model.add(Activation('softmax'))
    model.compile(loss='categorical_crossentropy', optimizer=Adadelta(), metrics=['accuracy'])
    models.append(model)
print models[0].summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_1 (Dense)              (None, 3)                 21        
_________________________________________________________________
activation_1 (Activation)    (None, 3)                 0         
_________________________________________________________________
dense_2 (Dense)              (None, 1)                 4         
_________________________________________________________________
activation_2 (Activation)    (None, 1)                 0         
_________________________________________________________________
dense_3 (Dense)              (None, 2)                 4         
_________________________________________________________________
activation_3 (Activation)    (None, 2)                 0         
Total params: 29.0
Trainable params: 29.0
Non-trainable params: 0.0
_________________________________________________________________
None


In [5]:
# Train models on Labelled Data
for ind in range(n_views):
    models[ind].fit(views[ind][0], views[ind][1], epochs=100, batch_size = 64, validation_split = 0.2, 
              callbacks=[EarlyStopping(monitor='val_acc', patience=5)])

Train on 4 samples, validate on 1 samples
Epoch 1/100
4/4 [==============================] - 0s - loss: 0.6931 - acc: 1.0000 - val_loss: 0.6927 - val_acc: 1.0000
Epoch 2/100
4/4 [==============================] - 0s - loss: 0.6927 - acc: 1.0000 - val_loss: 0.6922 - val_acc: 1.0000
Epoch 3/100
4/4 [==============================] - 0s - loss: 0.6922 - acc: 1.0000 - val_loss: 0.6918 - val_acc: 1.0000
Epoch 4/100
4/4 [==============================] - 0s - loss: 0.6918 - acc: 1.0000 - val_loss: 0.6913 - val_acc: 1.0000
Epoch 5/100
4/4 [==============================] - 0s - loss: 0.6913 - acc: 1.0000 - val_loss: 0.6909 - val_acc: 1.0000
Epoch 6/100
4/4 [==============================] - 0s - loss: 0.6909 - acc: 1.0000 - val_loss: 0.6904 - val_acc: 1.0000
Epoch 7/100
4/4 [==============================] - 0s - loss: 0.6904 - acc: 1.0000 - val_loss: 0.6899 - val_acc: 1.0000
Train on 4 samples, validate on 1 samples
Epoch 1/100
4/4 [==============================] - 0s - loss: 0.6931 - acc: 

In [6]:
# Run Co-Training Algorithm 1
# Simple majority voting over all the classifiers for the unlabelled example
L = views[0]
for ind in range(1, n_views):
    L[0] = np.concatenate([L[0], views[ind][0]], axis = 0)
    L[1] = np.concatenate([L[1], views[ind][1]], axis = 0)
changed = False
pred_modes = None
preds = np.zeros((U.shape[0], n_views))
for ind in range(n_views):
    preds[:, ind] = np.argmax(models[ind].predict(U), axis = 1)
while (not changed):
    pred_modes_new, counts = stats.mode(preds, axis=1)
    if pred_modes_new != pred_modes:
        changed = True
        pred_modes = pred_modes_new
        sel = np.array(np.argmax(pred_modes, axis=0), dtype=int)
        sel_one_hot = label_binarize([pred_modes[sel].squeeze()], classes=range(len(L_y[0]) + 1))[:, :-1]
        print sel_one_hot
        L[0] = np.concatenate([L[0], U[sel]], axis = 0)
        L[1] = np.concatenate([L[1], sel_one_hot], axis = 0)
    for ind in range(n_views):
        models[ind].fit(L[0], L[1], epochs=100, batch_size = 64, validation_split = 0.2, 
              callbacks=[EarlyStopping(monitor='val_acc', patience=5)])
perf = [None for _ in range(n_views)]
for ind in range(n_views):
    perf[ind] = model.evaluate(np.array(test_x), np.array(test_y))
    print perf[ind]

[[1 0]]
Train on 20 samples, validate on 6 samples
Epoch 1/100
20/20 [==============================] - 0s - loss: 0.6899 - acc: 1.0000 - val_loss: 0.6895 - val_acc: 1.0000
Epoch 2/100
20/20 [==============================] - 0s - loss: 0.6895 - acc: 1.0000 - val_loss: 0.6890 - val_acc: 1.0000
Epoch 3/100
20/20 [==============================] - 0s - loss: 0.6890 - acc: 1.0000 - val_loss: 0.6885 - val_acc: 1.0000
Epoch 4/100
20/20 [==============================] - 0s - loss: 0.6885 - acc: 1.0000 - val_loss: 0.6881 - val_acc: 1.0000
Epoch 5/100
20/20 [==============================] - 0s - loss: 0.6881 - acc: 1.0000 - val_loss: 0.6876 - val_acc: 1.0000
Epoch 6/100
20/20 [==============================] - 0s - loss: 0.6876 - acc: 1.0000 - val_loss: 0.6871 - val_acc: 1.0000
Epoch 7/100
20/20 [==============================] - 0s - loss: 0.6871 - acc: 1.0000 - val_loss: 0.6867 - val_acc: 1.0000
Train on 20 samples, validate on 6 samples
Epoch 1/100
20/20 [==============================] -

/usr/local/lib/python2.7/dist-packages/ipykernel/__main__.py:14: FutureWarning: comparison to `None` will result in an elementwise object comparison in the future.
